## Calculate visibilty from single point

In [ ]:
from objectnat import get_visibility_accurate, get_visibility
import geopandas as gpd
from shapely.geometry import Point

# Importing necessary libraries:
# - 'get_visibility_accurate' and 'get_visibility' from 'objectnat' to compute visibility.
# - 'geopandas' for working with geospatial data.
# - 'Point' from 'shapely.geometry' to define the viewpoint.

obstacles = gpd.read_parquet('examples_data/buildings.parquet').to_crs(32636)
# Loading building data from a .parquet file and reprojecting to the local coordinate system (EPSG:32636).

result1 = get_visibility(Point(348065, 6647674), obstacles, 500)
# Calculating visibility from a point (348065, 6647674) with a view radius of 500 units. 
# This method is faster but may "see behind buildings" and ignore certain occlusions.

result1 = gpd.GeoDataFrame(geometry=[result1], crs=32636)
# Converting the result of visibility analysis into a GeoDataFrame for easier manipulation and visualization.

result2 = get_visibility_accurate(Point(348065, 6647674), obstacles, 500)
# Calculating visibility using a more accurate method, which takes into account occlusions and bottlenecks,
# but at the cost of computational performance.

result2 = gpd.GeoDataFrame(geometry=[result2], crs=32636)
# Converting the accurate visibility result into a GeoDataFrame.

m1 = obstacles.explore(color='pink', tiles='CartoDB positron')
# Visualizing the obstacles (buildings) on an interactive map.
result2.explore(m=m1, color='blue')
# Overlaying the accurate visibility result (in blue) on the map with obstacles.

# result1.explore(m=m1, color='red')  # Uncomment this line to compare the faster method result (in red).

gpd.GeoDataFrame(geometry=[Point(348065, 6647674)], crs=32636).explore(m=m1, color='purple')
# Adding the viewpoint location (in purple) to the map for reference.

## Calculate visibilty from multiple points (multiproseccing)

In [ ]:
from objectnat import get_visibilities_from_points
import geopandas as gpd
from shapely.geometry import Point

# Importing 'get_visibilities_from_points' from 'objectnat' to calculate visibility from multiple points using multiprocessing.

obstacles = gpd.read_parquet('examples_data/buildings.parquet').to_crs(32636)
# Loading building data and reprojecting to the local coordinate system (EPSG:32636).

points = gpd.GeoDataFrame(geometry=[Point(348055, 6647684), Point(348175, 6647674), Point(348035, 6647634)], crs=32636)
# Defining multiple points from which visibility will be calculated.

result = get_visibilities_from_points(points, obstacles, 500)
# Calculating visibility from each point in the 'points' GeoDataFrame with a view distance of 500 units.
# This method uses multiprocessing for better performance when dealing with multiple points.

gpd.GeoDataFrame(geometry=result, crs=32636).explore()

## Calculate visibility catchment area (multiproseccing)

In [ ]:

import pandas as pd
import geopandas as gpd
from objectnat.methods.visibility_analysis import calculate_visibility_catchment_area

# Load data for buildings, points, woods, and bridges
builds = gpd.read_file('builds.geojson').to_crs(32636)
points = gpd.read_file('distributed_points.geojson').to_crs(32636)
woods = gpd.read_file('woods.geojson').to_crs(32636)
bridges = gpd.read_file('bridges.geojson').to_crs(32636)

view_dist = 1000
# Setting the visibility distance (catchment radius) to 1000 units.

obstacles = gpd.GeoDataFrame(pd.concat([builds, woods, bridges], ignore_index=True), geometry='geometry',
                             crs=32636)
# Combining the GeoDataFrames for buildings, woods, and bridges into a single GeoDataFrame that serves as obstacles 
# to be considered in the visibility calculation.

res = calculate_visibility_catchment_area(points, obstacles, view_dist)
# Calculating the visibility catchment area for the given points, considering the obstacles and the view distance.
# The result is a GeoDataFrame containing the catchment areas.

In [ ]:
res.explore(
    column="factor_normalized",
    categorical=False,
    cmap="plasma",
    legend=True,
)
# Visualizing the catchment areas on an interactive map, using the 'factor_normalized' column to color the areas
# with a 'plasma' colormap. A legend is displayed to show the range of values.